# Import package

In [1]:
import time
import requests
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 500)

import ast
import os
import pickle

from pandarallel import pandarallel

# Request to get the table of  all links

In [2]:
headers={
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36",
    "Connection": "keep-alive",
    "accept": "*/*",
    "Accept-Language": "zh-CN,zh;q=0.8",
    "content-type" : "application/json",
    "Host":  "www.aljazeera.com",
    "original-domain": "www.aljazeera.com",
    "Referer": "https://www.aljazeera.com/news",
    "wp-site": "aje",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Dest": "empty",
}

In [17]:
def crawl_urls(offset, tags):
    
    if tags == 'News':
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22news%22%2C%22categoryType%22%3A%22categories%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        
        headers['Referer'] = "https://www.aljazeera.com/news"
        
    elif tags == 'Conflicts':
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22conflict%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/conflict/"
        
    elif tags == 'Protests': 
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22protests%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/protests"
        
    elif tags == 'human-rights':     
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22human-rights%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/human-rights"

    elif tags == 'police':     
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22police%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/police"     
    
    elif tags == 'military':     
        url='https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22military%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/military"     
    
    elif tags == 'crime':     
        url= 'https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22crime%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/crime"  
    
    elif tags == 'gun-violence': 
        url= 'https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22gun-violence%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/gun-violence"  
        
    elif tags == 'politics': 
        url= 'https://www.aljazeera.com/graphql?wp-site=aje&operationName=AjeSectionPostsQuery&variables=%7B%22category%22%3A%22politics%22%2C%22categoryType%22%3A%22tags%22%2C%22postTypes%22%3A%5B%22blog%22%2C%22episode%22%2C%22opinion%22%2C%22post%22%2C%22video%22%2C%22external-article%22%2C%22gallery%22%2C%22podcast%22%5D%2C%22quantity%22%3A10%2C%22offset%22%3'\
            +'A%s'%offset +'%7D&extensions=%7B%7D'
        headers['Referer'] = "https://www.aljazeera.com/tag/politics"  
    

    res=requests.get(url, headers=headers)
    time.sleep(2)

    if res.status_code==200:
        js = res.json()
        return js['data']['articles']
    else:
        print('Error: %s'%offset)
        return

In [30]:
# crawl_urls(280, 'police')
# crawl_urls(600, 'military')
# crawl_urls(1400, 'crime')
# crawl_urls(450, 'gun-violence')

# df1 = [ [int('%s4'%i), 'police'] for i in range(1, 29)]
# df2 = [ [int('%s4'%i), 'military'] for i in range(1, 61)]
# df3 = [ [int('%s4'%i), 'crime' ] for i in range(1, 141)]
# df4 = [ [int('%s4'%i), 'gun-violence' ] for i in range(1, 46)]

# crawl_urls(1804, 'politics')

df1 = [ [int('%s4'%i), 'politics'] for i in range(1, 181)]
df = pd.DataFrame(df1, columns= ['offset', 'tags'])
df

,offset,tags
0,14,politics
1,24,politics
2,34,politics
3,44,politics
4,54,politics
...,...,...
175,1764,politics
176,1774,politics
177,1784,politics
178,1794,politics


In [31]:
pandarallel.initialize(nb_workers = 64, progress_bar = True)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
df[['rows']] = df.parallel_apply(lambda x: crawl_urls(x[0],x[1]), axis=1).values

In [33]:
df = df.explode('rows').reset_index(drop=True)
df = df.iloc[:,1:]
df

,tags,rows
0,politics,"{'id': '1376521', 'title': 'Murder of woman in Kyrgyzstan triggers bride kidnapping protests', 'excerpt': 'Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.', 'date': '2021-04-08T14:08:03', 'link': '/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/rd4w6', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', '..."
1,politics,"{'id': '1376494', 'title': '‘Sofagate’: EU-Turkey diplomatic scandal snowballs', 'excerpt': 'Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.', 'date': '2021-04-08T12:58:33', 'link': '/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/qpan8', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'au..."
2,politics,"{'id': '1376338', 'title': 'Northern Ireland unrest: Four key questions answered', 'excerpt': 'Long-simmering tensions have been brought to the boil by Brexit.', 'date': '2021-04-08T12:39:46', 'link': '/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/wt9k2', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'author': [], '__typename': 'Post', 'featuredImage': {'sourc..."
3,politics,"{'id': '1376436', 'title': 'A European take on US politics', 'excerpt': 'Former French ambassador to Washington gives assessment of global politics and offers advice to the Biden White House.', 'date': '2021-04-08T11:42:41', 'link': '/program/the-bottom-line/2021/4/8/a-european-take-on-us-politics', 'isBreaking': None, 'isLive': None, 'shortUrl': 'https://aje.io/3pp9k', 'isDeveloping': None, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'episode', 'author': [], '__typename': 'P..."
4,politics,"{'id': '1376139', 'title': 'Fierce contest for power as Samoa prepares for general election', 'excerpt': '‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.', 'date': '2021-04-08T04:26:51', 'link': '/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/kayw8', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postT..."
...,...,...
1785,politics,"{'id': '1127556', 'title': 'Jose Mujica: ‘I earn more than I need’', 'excerpt': 'The man described as the world’s ‘poorest president’ discusses Uruguay’s move to legalise marijuana.', 'date': '2013-10-26T15:00:26', 'link': '/program/talk-to-al-jazeera/2013/10/26/jose-mujica-i-earn-more-than-i-need', 'isBreaking': None, 'isLive': None, 'shortUrl': 'https://aje.io/5sbdx', 'isDeveloping': None, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'episode', 'author': [], '__typename': 'P..."
1786,politics,"{'id': '1127416', 'title': 'Mahinda Rajapaksa: ‘This is all propaganda’', 'excerpt': 'Sri Lanka’s president discusses a recent UN report accusing him of pushing the country into an authoritarian direction.', 'date': '2013-09-29T12:39:44', 'link': '/program/talk-to-al-jazeera/2013/9/29/mahinda-rajapaksa-this-is-all-propaganda', 'isBreaking': None, 'isLive': None, 'shortUrl': 'https://aje.io/yk3h9', 'isDeveloping': None, 'externalUrl': None, 'replacementHeadline': 'Interview with Rajapaksa', ..."
1787,politics,"{'id': '1126871', 'title': 'Moaz al-Khatib: The priority is to save Syria', 'excerpt': 'Outgoing head of the Syrian National Coalition discusses his decision to resign and the next steps for the opposition.', 'date': '2013-05-11T09:59:58', 'link': '/program/talk-to-al-jazeera/2013/5/11/moaz-al-khatib-the-priority-is-to-save-syria', 'isBreaking': None, 'isLive': No

In [34]:
df = df[~df.rows.isnull()]
df

,tags,rows
0,politics,"{'id': '1376521', 'title': 'Murder of woman in Kyrgyzstan triggers bride kidnapping protests', 'excerpt': 'Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.', 'date': '2021-04-08T14:08:03', 'link': '/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/rd4w6', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', '..."
1,politics,"{'id': '1376494', 'title': '‘Sofagate’: EU-Turkey diplomatic scandal snowballs', 'excerpt': 'Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.', 'date': '2021-04-08T12:58:33', 'link': '/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/qpan8', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'au..."
2,politics,"{'id': '1376338', 'title': 'Northern Ireland unrest: Four key questions answered', 'excerpt': 'Long-simmering tensions have been brought to the boil by Brexit.', 'date': '2021-04-08T12:39:46', 'link': '/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/wt9k2', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'author': [], '__typename': 'Post', 'featuredImage': {'sourc..."
3,politics,"{'id': '1376436', 'title': 'A European take on US politics', 'excerpt': 'Former French ambassador to Washington gives assessment of global politics and offers advice to the Biden White House.', 'date': '2021-04-08T11:42:41', 'link': '/program/the-bottom-line/2021/4/8/a-european-take-on-us-politics', 'isBreaking': None, 'isLive': None, 'shortUrl': 'https://aje.io/3pp9k', 'isDeveloping': None, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'episode', 'author': [], '__typename': 'P..."
4,politics,"{'id': '1376139', 'title': 'Fierce contest for power as Samoa prepares for general election', 'excerpt': '‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.', 'date': '2021-04-08T04:26:51', 'link': '/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/kayw8', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postT..."
...,...,...
1784,politics,"{'id': '1106982', 'title': 'Mandela mystique clings to Soweto streets', 'excerpt': 'In areas of Johannesburg where Mandela forged his political career, his presence was strong but not always enough.', 'date': '2013-12-05T23:21:30', 'link': '/features/2013/12/5/mandela-mystique-clings-to-soweto-streets', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/jjcrt', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': 'Soweto’s Mandela mystique', 'postType': 'post', '..."
1785,politics,"{'id': '1127556', 'title': 'Jose Mujica: ‘I earn more than I need’', 'excerpt': 'The man described as the world’s ‘poorest president’ discusses Uruguay’s move to legalise marijuana.', 'date': '2013-10-26T15:00:26', 'link': '/program/talk-to-al-jazeera/2013/10/26/jose-mujica-i-earn-more-than-i-need', 'isBreaking': None, 'isLive': None, 'shortUrl': 'https://aje.io/5sbdx', 'isDeveloping': None, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'episode', 'author': [], '__typename': 'P..."
1786,politics,"{'id': '1127416', 'title': 'Mahinda Rajapaksa: ‘This is all propaganda’', 'excerpt': 'Sri Lanka’s president discusses a recent UN report accusing him of pushing the country into an authoritarian direction.', 'date': '2013-09-29T12:39:44', 'link': '/program/talk-to-al-jazeera/2013/9/29/mahinda-rajapaksa-this-is-all-propaganda', 'isBreaking': None, 'isLive': None, 

In [13]:
# df.to_csv('alijazeera_raw_conflict.csv', index = False)

# Get link and abstract per story

In [2]:
# df = pd.read_csv('alijazeera_raw_conflict.csv', header= [0])
# df

,tags,rows
0,Conflicts,"{'id': '1339490', 'title': 'Turkey summons Iran envoy over remarks on Iraq operations', 'excerpt': 'Turkey says the PKK presence in Iraq is a national security threat and it is Baghdad’s responsibility to take action.', 'date': '2021-02-28T12:54:13', 'link': '/news/2021/2/28/turkey-summons-iran-envoy-over-remarks-on-iraq-operations', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/4zv7w', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'po..."
1,Conflicts,"{'id': '1339353', 'title': 'Israeli-owned vessel docked in Dubai after mysterious explosion', 'excerpt': 'American officials say the vessel sustained two holes on its port side and two on its starboard side.', 'date': '2021-02-28T11:00:37', 'link': '/news/2021/2/28/israeli-owned-cargo-docked-in-dubai-after-mysterious-explosion', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/7cyld', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', ..."
2,Conflicts,"{'id': '1339257', 'title': 'Analysis: Syria’s peace process and the Russian and US roles', 'excerpt': 'Ten years into Syria’s devastating war, the country’s peace prospects have never been gloomier.', 'date': '2021-02-28T09:00:03', 'link': '/news/2021/2/28/analysis-syrias-peace-process-and-the-russian-and-us-role', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/vdtxb', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'author': [{'i..."
3,Conflicts,"{'id': '1339274', 'title': 'Fighting intensifies between Houthis, Yemen gov’t forces in Marib', 'excerpt': 'Dozens of fighters from both sides killed in ‘unabated’ fighting since Friday in Yemen’s strategic province of Marib.', 'date': '2021-02-28T08:08:11', 'link': '/news/2021/2/28/fighting-intensifies-between-houthis-yemen-govt-forces-in-marib', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/tcnpz', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', '..."
4,Conflicts,"{'id': '1337291', 'title': 'Killings in Axum by Eritrea troops ‘may amount to war crimes’', 'excerpt': 'Amnesty says hundreds of civilians were killed by Eritrean troops in Ethiopian town of Axum last November.', 'date': '2021-02-26T00:04:30', 'link': '/news/2021/2/26/killings-in-axum-by-eritrea-may-amount-to-war-crimes-amnesty', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/phlmx', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post',..."
...,...,...
15028,human-rights,"{'id': '988596', 'title': 'Release: Frost over the World', 'excerpt': 'Al Jazeera English announces “Frost Over The World” programme, with award winning media legend Sir David Frost', 'date': '2006-12-13T07:10:31', 'link': '/features/2006/12/13/release-frost-over-the-world-2', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/lgy6t', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'author': [], '__typename': 'Post', 'featuredImage': ..."
15029,human-rights,"{'id': '988592', 'title': 'Release: Frost Over The World', 'excerpt': 'Al Jazeera English announces the line-up for today’s “Frost Over The World”', 'date': '2006-12-13T07:09:25', 'link': '/features/2006/12/13/release-frost-over-the-world', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/ue6zp', 'isDeveloping': False, 'externalUrl': None, 'replacementHeadline': '', 'postType': 'post', 'author': [], '__typename': 'Post', 'featuredImage': {'sourceUrl': '/wp-content/uploads/20..."
15030,human-rights,"{'id': '988542', 'title': 'Zoo poisons lion cubs to cut costs', 'excerpt': 'Stuffed bodies of Ethiopia’s rare black-maned lion cubs are sold for $400.', 'date': '2006-11-22T19:33:57', 'link': '/news/2006/11/22/zoo-poisons-lion-cubs-to-cut-costs', 'isBreaking': False, 'isLive': False, 'shortUrl': 'https://aje.io/favcb', 'isDeveloping': False, '

In [35]:
new_cols = ['id','title','excerpt','date','link', 'postType']
# df1 = df['rows'].apply(ast.literal_eval).apply(pd.Series)[new_cols]
df1 = df['rows'].apply(pd.Series)[new_cols]
df1 = pd.concat([df[['tags']], df1], axis=1,  join='outer')
df1 = df1[~df1.link.isnull()]
df1.link = df1.link.apply(lambda x: 'https://www.aljazeera.com' + x if x.startswith('/') else x)
df1

,tags,id,title,excerpt,date,link,postType
0,politics,1376521,Murder of woman in Kyrgyzstan triggers bride kidnapping protests,Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.,2021-04-08T14:08:03,https://www.aljazeera.com/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan,post
1,politics,1376494,‘Sofagate’: EU-Turkey diplomatic scandal snowballs,"Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.",2021-04-08T12:58:33,https://www.aljazeera.com/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore,post
2,politics,1376338,Northern Ireland unrest: Four key questions answered,Long-simmering tensions have been brought to the boil by Brexit.,2021-04-08T12:39:46,https://www.aljazeera.com/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered,post
3,politics,1376436,A European take on US politics,Former French ambassador to Washington gives assessment of global politics and offers advice to the Biden White House.,2021-04-08T11:42:41,https://www.aljazeera.com/program/the-bottom-line/2021/4/8/a-european-take-on-us-politics,episode
4,politics,1376139,Fierce contest for power as Samoa prepares for general election,‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.,2021-04-08T04:26:51,https://www.aljazeera.com/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election,post
...,...,...,...,...,...,...,...
1784,politics,1106982,Mandela mystique clings to Soweto streets,"In areas of Johannesburg where Mandela forged his political career, his presence was strong but not always enough.",2013-12-05T23:21:30,https://www.aljazeera.com/features/2013/12/5/mandela-mystique-clings-to-soweto-streets,post
1785,politics,1127556,Jose Mujica: ‘I earn more than I need’,The man described as the world’s ‘poorest president’ discusses Uruguay’s move to legalise marijuana.,2013-10-26T15:00:26,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/10/26/jose-mujica-i-earn-more-than-i-need,episode
1786,politics,1127416,Mahinda Rajapaksa: ‘This is all propaganda’,Sri Lanka’s president discusses a recent UN report accusing him of pushing the country into an authoritarian direction.,2013-09-29T12:39:44,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/9/29/mahinda-rajapaksa-this-is-all-propaganda,episode
1787,politics,1126871,Moaz al-Khatib: The priority is to save Syria,Outgoing head of the Syrian National Coalition discusses his decision to resign and the next steps for the opposition.,2013-05-11T09:59:58,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/5/11/moaz-al-khatib-the-priority-is-to-save-syria,episode


In [36]:
df1[df1.id.duplicated(keep=False)].sort_values(by=['id'])

,tags,id,title,excerpt,date,link,postType


In [37]:
df_others = pd.read_csv('alijazeera_text.csv', header=[0])
df_others

,id,title,excerpt,date,link,postType,text
0,1339005,Ecuador health minister steps down over COVID vaccine roll-out,Juan Carlos Zevallos is latest government official in South America to resign over coronavirus vaccine scandals.,2021-02-27T17:00:06,https://www.aljazeera.com/news/2021/2/27/ecuador-health-minister-steps-down-over-covid-vaccine-roll-out,post,"Ecuador’s health minister has stepped down after criticism over the distribution plan of COVID-19 vaccines, the latest vaccination-related scandal to roil governments in South America.\nJuan Carlos Zevallos tendered his resignation on Friday after claims of irregularities in a pilot programme for vaccinations that the government has been carrying out since January.\nZevallos is under investigation by state prosecutors for mishandling the roll-out after he participated in an inoculation effor..."
1,1338999,Armenian president rejects army chief’s dismissal after PM order,PM Nikol Pashinyan had urged President Armen Sarkissian to fire army’s chief of staff after accusing military of coup.,2021-02-27T16:54:24,https://www.aljazeera.com/news/2021/2/27/armenian-president-rejects-army-chiefs-dismissal,post,"Armenian President Armen Sarkissian has refused to sign a prime ministerial order to dismiss the army’s chief of staff, deepening a national political crisis.\nThe country has faced turmoil since Prime Minister Nikol Pashinyan signed a Russian-brokered peace accord in November, sealing a humiliating defeat to Azerbaijan after six weeks of fierce fighting over the disputed Nagorno-Karabakh region.\nDivisions widened on Thursday when Pashinyan defied a call by the military to resign, accusing ..."
2,1332409,Inuit voices grow louder in fight over Nunavut mine expansion,Inuit communities in northern Canada say they fear iron ore mine expansion will harm wildlife and cultural practices.,2021-02-27T15:40:47,https://www.aljazeera.com/news/2021/2/27/inuit-voices-grow-louder-in-fight-over-nunavut-mine-expansion-canada,post,"Montreal, Canada – As the mayor of Clyde River, a small Inuit hamlet in the territory of Nunavut in northern Canada, Jerry Natanine had seen local opposition to an iron ore mining project building for years.\nSo, when a group of Inuit hunters this month set up a protest blockade at the Mary River mine on Baffin Island, in the Arctic Archipelago, he was not surprised. “They’re not here to listen to us at all,” Natanine told Al Jazeera in a phone interview, referring to Baffinland Iron Mines C..."
3,1338965,Thousands mark anniversary of Kremlin critic Nemtsov’s murder,Opposition supporters laid flowers on a bridge in Moscow where Boris Nemtsov was killed six years ago.,2021-02-27T15:29:21,https://www.aljazeera.com/news/2021/2/27/russian-opposition-marks-anniversary-of-kremlin-critic-nemtsov,post,"Several thousand people commemorated the anniversary of the 2015 murder of Russian opposition politician Boris Nemtsov in central Moscow on Saturday, following a clampdown on protests over the jailing of Kremlin critic Alexey Navalny.\nWith a minimal police presence at the scene, opposition supporters laid flowers on the bridge in central Moscow where Nemtsov was gunned down on the night of February 27 six years ago.\nNemtsov briefly served as deputy prime minister in the late 1990s before j..."
4,1338959,Tanzania COVID U-turn ‘a good move’ – but is it good enough?,"After months of downplaying deadly disease, government officials have urged people to take precautions and wear masks.",2021-02-27T14:45:27,https://www.aljazeera.com/news/2021/2/27/tanzania-covid-uturn-good-move-but-good-enough,post,"Dar-es-Salaam, Tanzania – Esther Mngodo, like other Tanzanians, was relieved to hear this week government officials are finally urging people in the country to take precautions against the coronavirus – and even wear face masks.\n“It is a good move,” said Mngodo, a 34-year-old resident of Dar-es-Salaam. “But much more needs to be done to increase public awareness, testing and treatment. M

In [38]:
df1.shape, df1[df1.link.isin(df_others.link)].shape

((1789, 7), (1104, 7))

In [40]:
df1 = df1[~df1.link.isin(df_others.link)].reset_index(drop=True)
df1

,tags,id,title,excerpt,date,link,postType
0,politics,1376521,Murder of woman in Kyrgyzstan triggers bride kidnapping protests,Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.,2021-04-08T14:08:03,https://www.aljazeera.com/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan,post
1,politics,1376494,‘Sofagate’: EU-Turkey diplomatic scandal snowballs,"Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.",2021-04-08T12:58:33,https://www.aljazeera.com/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore,post
2,politics,1376338,Northern Ireland unrest: Four key questions answered,Long-simmering tensions have been brought to the boil by Brexit.,2021-04-08T12:39:46,https://www.aljazeera.com/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered,post
3,politics,1376436,A European take on US politics,Former French ambassador to Washington gives assessment of global politics and offers advice to the Biden White House.,2021-04-08T11:42:41,https://www.aljazeera.com/program/the-bottom-line/2021/4/8/a-european-take-on-us-politics,episode
4,politics,1376139,Fierce contest for power as Samoa prepares for general election,‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.,2021-04-08T04:26:51,https://www.aljazeera.com/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election,post
...,...,...,...,...,...,...,...
680,politics,1106982,Mandela mystique clings to Soweto streets,"In areas of Johannesburg where Mandela forged his political career, his presence was strong but not always enough.",2013-12-05T23:21:30,https://www.aljazeera.com/features/2013/12/5/mandela-mystique-clings-to-soweto-streets,post
681,politics,1127556,Jose Mujica: ‘I earn more than I need’,The man described as the world’s ‘poorest president’ discusses Uruguay’s move to legalise marijuana.,2013-10-26T15:00:26,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/10/26/jose-mujica-i-earn-more-than-i-need,episode
682,politics,1127416,Mahinda Rajapaksa: ‘This is all propaganda’,Sri Lanka’s president discusses a recent UN report accusing him of pushing the country into an authoritarian direction.,2013-09-29T12:39:44,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/9/29/mahinda-rajapaksa-this-is-all-propaganda,episode
683,politics,1126871,Moaz al-Khatib: The priority is to save Syria,Outgoing head of the Syrian National Coalition discusses his decision to resign and the next steps for the opposition.,2013-05-11T09:59:58,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/5/11/moaz-al-khatib-the-priority-is-to-save-syria,episode


In [41]:
df1 = df1[~df1.id.duplicated()].reset_index(drop=True)
df1 = df1.iloc[:,1:]
df1

,id,title,excerpt,date,link,postType
0,1376521,Murder of woman in Kyrgyzstan triggers bride kidnapping protests,Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.,2021-04-08T14:08:03,https://www.aljazeera.com/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan,post
1,1376494,‘Sofagate’: EU-Turkey diplomatic scandal snowballs,"Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.",2021-04-08T12:58:33,https://www.aljazeera.com/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore,post
2,1376338,Northern Ireland unrest: Four key questions answered,Long-simmering tensions have been brought to the boil by Brexit.,2021-04-08T12:39:46,https://www.aljazeera.com/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered,post
3,1376436,A European take on US politics,Former French ambassador to Washington gives assessment of global politics and offers advice to the Biden White House.,2021-04-08T11:42:41,https://www.aljazeera.com/program/the-bottom-line/2021/4/8/a-european-take-on-us-politics,episode
4,1376139,Fierce contest for power as Samoa prepares for general election,‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.,2021-04-08T04:26:51,https://www.aljazeera.com/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election,post
...,...,...,...,...,...,...
680,1106982,Mandela mystique clings to Soweto streets,"In areas of Johannesburg where Mandela forged his political career, his presence was strong but not always enough.",2013-12-05T23:21:30,https://www.aljazeera.com/features/2013/12/5/mandela-mystique-clings-to-soweto-streets,post
681,1127556,Jose Mujica: ‘I earn more than I need’,The man described as the world’s ‘poorest president’ discusses Uruguay’s move to legalise marijuana.,2013-10-26T15:00:26,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/10/26/jose-mujica-i-earn-more-than-i-need,episode
682,1127416,Mahinda Rajapaksa: ‘This is all propaganda’,Sri Lanka’s president discusses a recent UN report accusing him of pushing the country into an authoritarian direction.,2013-09-29T12:39:44,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/9/29/mahinda-rajapaksa-this-is-all-propaganda,episode
683,1126871,Moaz al-Khatib: The priority is to save Syria,Outgoing head of the Syrian National Coalition discusses his decision to resign and the next steps for the opposition.,2013-05-11T09:59:58,https://www.aljazeera.com/program/talk-to-al-jazeera/2013/5/11/moaz-al-khatib-the-priority-is-to-save-syria,episode


Then we choose post and gallery

In [42]:
df1 = df1[df1.postType.isin(['post', 'gallery'])].reset_index(drop=True)
df1

,id,title,excerpt,date,link,postType
0,1376521,Murder of woman in Kyrgyzstan triggers bride kidnapping protests,Protesters call for interior minister’s resignation after the kidnapping and killing of 27-year-old Aizada Kanatbekova.,2021-04-08T14:08:03,https://www.aljazeera.com/news/2021/4/8/protests-break-out-after-murder-of-abducted-woman-in-kyrgyzstan,post
1,1376494,‘Sofagate’: EU-Turkey diplomatic scandal snowballs,"Anger grows after male leaders pictured taking chairs, leaving von der Leyen standing before being ushered to a sofa.",2021-04-08T12:58:33,https://www.aljazeera.com/news/2021/4/8/eu-legislators-demand-answers-over-turkey-sofagate-furore,post
2,1376338,Northern Ireland unrest: Four key questions answered,Long-simmering tensions have been brought to the boil by Brexit.,2021-04-08T12:39:46,https://www.aljazeera.com/news/2021/4/8/northern-ireland-unrest-four-key-questions-answered,post
3,1376139,Fierce contest for power as Samoa prepares for general election,‘Different mood’ in Samoa as a former deputy challenges Prime Minister Tuilaepa Sailele Malielegaoi’s 23-year rule.,2021-04-08T04:26:51,https://www.aljazeera.com/news/2021/4/8/fierce-contest-for-power-as-samoa-prepares-for-general-election,post
4,1376130,South Korea: Moon’s party dealt crushing defeats in mayoral polls,Conservative opposition won landslide victories for the mayors of the country’s two biggest cities of Seoul and Busan.,2021-04-08T03:46:20,https://www.aljazeera.com/news/2021/4/8/moons-ruling-party-dealt-crushing-poll-defeats-in-south-korea,post
...,...,...,...,...,...,...
291,1107865,Nepal’s journey from war to peace,A new book looks at Himalayan nation’s decades of political change and difficult transition from monarchy to democracy.,2014-07-20T10:24:53,https://www.aljazeera.com/features/2014/7/20/nepals-journey-from-war-to-peace,post
292,1163171,In Pictures: South Africa’s massive strikes,South Africa braces for one of the largest labour strikes over wages in the country’s history.,2014-07-05T09:06:27,https://www.aljazeera.com/gallery/2014/7/5/in-pictures-south-africas-massive-strikes,gallery
293,1163152,In Pictures: Cambodian exodus from Thailand,Security forces appear to be deporting Cambodian migrants in what activists say is a well coordinated campaign.,2014-06-20T12:42:42,https://www.aljazeera.com/gallery/2014/6/20/in-pictures-cambodian-exodus-from-thailand,gallery
294,1107289,Mandela: ‘A man of the people’,South African photojournalist Louise Gubb shares the story of the Mandela she knew – from behind and beyond the lens.,2014-02-10T10:23:52,https://www.aljazeera.com/features/2014/2/10/mandela-a-man-of-the-people,post


In [13]:
# df1.to_csv('alijazeera_relevant_conflict.csv', index = False)

In [43]:
# df_duplicate =  pd.merge(df_org, df1, how = 'inner')
# df1_not_duplicated = pd.concat([df1, df_duplicate]).drop_duplicates(keep=False).reset_index(drop=True)

df1_not_duplicated = df1

In [44]:
def extract_url(url, postType):
    time.sleep(2)
    try:
        coverpage = requests.get(url).content
        soup = BeautifulSoup(coverpage, 'html.parser')
    except:
        print('exception', url)
        return 'exception'
    find_class ='wysiwyg wysiwyg--all-content' if postType =='post' else 'gallery-content wysiwyg wysiwyg--all-content'
    
    find_text = soup.find_all('div', class_= find_class)
    
    if len(find_text)==0:
        print('find none', url)
        return 'none'

    paragraphs = find_text[0].find_all('p', recursive= False)
    text = [i.text for i in paragraphs]
    text = '\n'.join(text)
    return text

In [45]:
pandarallel.initialize(nb_workers= 128, progress_bar=True)
df1_not_duplicated[['text']] = df1_not_duplicated[['link', 'postType']].parallel_apply(lambda x: extract_url(x[0],x[1]), axis=1).values

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [46]:
df1_not_duplicated.to_csv('alijazeera_text_add_politics.csv', index = False)

In [93]:
df1_not_duplicated = pd.read_csv('alijazeera_text_add_politics.csv', header = [0])

In [47]:
df_org = pd.read_csv('alijazeera_text.csv', header = [0])

In [48]:
df1_not_duplicated.text.str.split().str.len()

0       473
1       435
2      1005
3      1151
4       471
       ... 
291    1673
292     178
293     125
294    1192
295    1384
Name: text, Length: 296, dtype: int64

In [49]:
df_org.text.str.split().str.len()

0          472.0
1          401.0
2         1616.0
3          522.0
4          897.0
           ...  
165311    1743.0
165312     733.0
165313     406.0
165314    1144.0
165315     429.0
Name: text, Length: 165316, dtype: float64

In [53]:
df_finall = pd.concat([df_org, df1_not_duplicated]).reset_index(drop=True)
# df_finall.to_csv('alijazeera_relevant_conflict_finall.csv', index = False)
df_finall

,id,title,excerpt,date,link,postType,text
0,1339005,Ecuador health minister steps down over COVID vaccine roll-out,Juan Carlos Zevallos is latest government official in South America to resign over coronavirus vaccine scandals.,2021-02-27T17:00:06,https://www.aljazeera.com/news/2021/2/27/ecuador-health-minister-steps-down-over-covid-vaccine-roll-out,post,"Ecuador’s health minister has stepped down after criticism over the distribution plan of COVID-19 vaccines, the latest vaccination-related scandal to roil governments in South America.\nJuan Carlos Zevallos tendered his resignation on Friday after claims of irregularities in a pilot programme for vaccinations that the government has been carrying out since January.\nZevallos is under investigation by state prosecutors for mishandling the roll-out after he participated in an inoculation effor..."
1,1338999,Armenian president rejects army chief’s dismissal after PM order,PM Nikol Pashinyan had urged President Armen Sarkissian to fire army’s chief of staff after accusing military of coup.,2021-02-27T16:54:24,https://www.aljazeera.com/news/2021/2/27/armenian-president-rejects-army-chiefs-dismissal,post,"Armenian President Armen Sarkissian has refused to sign a prime ministerial order to dismiss the army’s chief of staff, deepening a national political crisis.\nThe country has faced turmoil since Prime Minister Nikol Pashinyan signed a Russian-brokered peace accord in November, sealing a humiliating defeat to Azerbaijan after six weeks of fierce fighting over the disputed Nagorno-Karabakh region.\nDivisions widened on Thursday when Pashinyan defied a call by the military to resign, accusing ..."
2,1332409,Inuit voices grow louder in fight over Nunavut mine expansion,Inuit communities in northern Canada say they fear iron ore mine expansion will harm wildlife and cultural practices.,2021-02-27T15:40:47,https://www.aljazeera.com/news/2021/2/27/inuit-voices-grow-louder-in-fight-over-nunavut-mine-expansion-canada,post,"Montreal, Canada – As the mayor of Clyde River, a small Inuit hamlet in the territory of Nunavut in northern Canada, Jerry Natanine had seen local opposition to an iron ore mining project building for years.\nSo, when a group of Inuit hunters this month set up a protest blockade at the Mary River mine on Baffin Island, in the Arctic Archipelago, he was not surprised. “They’re not here to listen to us at all,” Natanine told Al Jazeera in a phone interview, referring to Baffinland Iron Mines C..."
3,1338965,Thousands mark anniversary of Kremlin critic Nemtsov’s murder,Opposition supporters laid flowers on a bridge in Moscow where Boris Nemtsov was killed six years ago.,2021-02-27T15:29:21,https://www.aljazeera.com/news/2021/2/27/russian-opposition-marks-anniversary-of-kremlin-critic-nemtsov,post,"Several thousand people commemorated the anniversary of the 2015 murder of Russian opposition politician Boris Nemtsov in central Moscow on Saturday, following a clampdown on protests over the jailing of Kremlin critic Alexey Navalny.\nWith a minimal police presence at the scene, opposition supporters laid flowers on the bridge in central Moscow where Nemtsov was gunned down on the night of February 27 six years ago.\nNemtsov briefly served as deputy prime minister in the late 1990s before j..."
4,1338959,Tanzania COVID U-turn ‘a good move’ – but is it good enough?,"After months of downplaying deadly disease, government officials have urged people to take precautions and wear masks.",2021-02-27T14:45:27,https://www.aljazeera.com/news/2021/2/27/tanzania-covid-uturn-good-move-but-good-enough,post,"Dar-es-Salaam, Tanzania – Esther Mngodo, like other Tanzanians, was relieved to hear this week government officials are finally urging people in the country to take precautions against the coronavirus – and even wear face masks.\n“It is a good move,” said Mngodo, a 34-year-old resident of Dar-es-Salaam. “But much more needs to be done to increase public awareness, testing and treatment. M

In [54]:
print(df_finall.shape)
df_finall = df_finall[df_finall.text.str.len()>200]
print(df_finall.shape)
df_finall = df_finall[~df_finall.duplicated(subset='text')]
print(df_finall.shape)

(165612, 7)
(149400, 7)
(149236, 7)


In [58]:
df_finall = df_finall[['link','date','title','excerpt','text']]
df_finall.columns=['url','date','title','abstract','text']
df_finall

,url,date,title,abstract,text
0,https://www.aljazeera.com/news/2021/2/27/ecuador-health-minister-steps-down-over-covid-vaccine-roll-out,2021-02-27T17:00:06,Ecuador health minister steps down over COVID vaccine roll-out,Juan Carlos Zevallos is latest government official in South America to resign over coronavirus vaccine scandals.,"Ecuador’s health minister has stepped down after criticism over the distribution plan of COVID-19 vaccines, the latest vaccination-related scandal to roil governments in South America.\nJuan Carlos Zevallos tendered his resignation on Friday after claims of irregularities in a pilot programme for vaccinations that the government has been carrying out since January.\nZevallos is under investigation by state prosecutors for mishandling the roll-out after he participated in an inoculation effor..."
1,https://www.aljazeera.com/news/2021/2/27/armenian-president-rejects-army-chiefs-dismissal,2021-02-27T16:54:24,Armenian president rejects army chief’s dismissal after PM order,PM Nikol Pashinyan had urged President Armen Sarkissian to fire army’s chief of staff after accusing military of coup.,"Armenian President Armen Sarkissian has refused to sign a prime ministerial order to dismiss the army’s chief of staff, deepening a national political crisis.\nThe country has faced turmoil since Prime Minister Nikol Pashinyan signed a Russian-brokered peace accord in November, sealing a humiliating defeat to Azerbaijan after six weeks of fierce fighting over the disputed Nagorno-Karabakh region.\nDivisions widened on Thursday when Pashinyan defied a call by the military to resign, accusing ..."
2,https://www.aljazeera.com/news/2021/2/27/inuit-voices-grow-louder-in-fight-over-nunavut-mine-expansion-canada,2021-02-27T15:40:47,Inuit voices grow louder in fight over Nunavut mine expansion,Inuit communities in northern Canada say they fear iron ore mine expansion will harm wildlife and cultural practices.,"Montreal, Canada – As the mayor of Clyde River, a small Inuit hamlet in the territory of Nunavut in northern Canada, Jerry Natanine had seen local opposition to an iron ore mining project building for years.\nSo, when a group of Inuit hunters this month set up a protest blockade at the Mary River mine on Baffin Island, in the Arctic Archipelago, he was not surprised. “They’re not here to listen to us at all,” Natanine told Al Jazeera in a phone interview, referring to Baffinland Iron Mines C..."
3,https://www.aljazeera.com/news/2021/2/27/russian-opposition-marks-anniversary-of-kremlin-critic-nemtsov,2021-02-27T15:29:21,Thousands mark anniversary of Kremlin critic Nemtsov’s murder,Opposition supporters laid flowers on a bridge in Moscow where Boris Nemtsov was killed six years ago.,"Several thousand people commemorated the anniversary of the 2015 murder of Russian opposition politician Boris Nemtsov in central Moscow on Saturday, following a clampdown on protests over the jailing of Kremlin critic Alexey Navalny.\nWith a minimal police presence at the scene, opposition supporters laid flowers on the bridge in central Moscow where Nemtsov was gunned down on the night of February 27 six years ago.\nNemtsov briefly served as deputy prime minister in the late 1990s before j..."
4,https://www.aljazeera.com/news/2021/2/27/tanzania-covid-uturn-good-move-but-good-enough,2021-02-27T14:45:27,Tanzania COVID U-turn ‘a good move’ – but is it good enough?,"After months of downplaying deadly disease, government officials have urged people to take precautions and wear masks.","Dar-es-Salaam, Tanzania – Esther Mngodo, like other Tanzanians, was relieved to hear this week government officials are finally urging people in the country to take precautions against the coronavirus – and even wear face masks.\n“It is a good move,” said Mngodo, a 34-year-old resident of Dar-es-Salaam. “But much more needs to be done to increase public awareness, testing and treatment. Most importantly, we must have a clear strategy on how to navigate through the

In [59]:
df_finall.to_csv('alijazeera.csv', index= False)

In [3]:
df_finall = pd.read_csv('alijazeera.csv', header=[0])
df_finall.sort_values('date')

,url,date,title,abstract,text
148940,https://www.aljazeera.com/news/2003/4/29/seven-us-soldiers-injured-in-falluja-attack,2003-04-29T06:56:48,Seven US soldiers injured in Falluja attack,Seven US soldiers wounded in Iraq grenade attack,"\n\n\n\n\n\n\n\n\n\nVictim of US firing: grenade attack\non soldiers seen as angry reaction\n\n\n\n\n\nSeven US soldiers were wounded in a grenade attack at their base in the Iraqi city of Falluja, where American troops killed at least 15 people during protests this week, the US military said on Thursday. \n \n“The attack was an expression of the anger of a few people in the city after what happened,” Captain Alan Vaught, the local commander of the troops, said in reference to the US shootin..."
148939,https://www.aljazeera.com/news/2003/4/29/arab-economies-of-us-national-interest,2003-04-29T08:09:44,Arab economies of US national interest,US seeks to increase trade to spur Arab economic growth but daunting challenges need to be overcome,"The United States is seeking to broaden its trade ties with the Arab world, including the possibility of an ambitious Middle East trade initiative aimed at linking several Arab countries into free trade agreements with the United States.\nThe idea, touched on by US Trade Representative Robert Zoellick at a recent Washington conference, could potentially emulate a recently launched US programme aimed at boosting US trade ties with southeast Asian nations. Under that model, the US seeks to cre..."
148938,https://www.aljazeera.com/news/2003/4/29/sahaf-offered-tv-post,2003-04-29T10:52:37,Sahaf offered TV post,The former Iraqi information minister has been offered an analyst position with Dubai-based news channel Al Arabiyya,"\nAn Arab television network said on Tuesday it wants to give a job to former Iraqi information minister Mohammed Saeed al-Sahaf, whose colourful daily briefings during the US-led invasion won him a cult following. \n\n\nAli al-Hadethi, supervisor of the Dubai-based Al Arabiyya satellite channel, told Reuters that Sahaf, who does not figure on Washington’s list of 55 most-wanted Iraqis, was welcome to join the network immediately as a commentator and analyst. \n\n\nHadethi said he did not kn..."
148937,https://www.aljazeera.com/news/2003/4/29/oil-stunts-middle-east-economies,2003-04-29T17:30:03,Oil stunts Middle East economies,Oil may have brought untold riches to the Middle East but on its own it is a tenuous and ineffective base for regional development,"The Middle East, which sits on 64 percent of the world’s oil resources, has ridden an oil wave for at least the last three decades in the belief that it will bring in much-needed cash for the region’s economies.\n\n \n\nHowever, the abundance of black gold seems to have diverted attention from other sectors of the Middle East economy. Analysts point out that the easy money oil has brought in appears to have sidelined other important priorities in the region. \n \nNon-oil-related industries h..."
148936,https://www.aljazeera.com/news/2003/4/29/the-lesson-from-lebanon,2003-04-29T20:26:40,The lesson from Lebanon,"Viewed from Beirut, the war in Iraq is painfully familiar. War is a bloody, destructive business everywhere outside Hollywood","Eighteen different religious communities and several different ethnicities existing in sometimes uneasy equilibrium, squeezed into a mountainous country 200 km long and less than 50 km wide. On paper, Lebanon doesn’t look like it ought to work. In reality, it sometimes doesn’t. \n \nThe last, and most spectacular time it failed to do so, the country was plunged into almost two decades of fratricidal wars that pitted community against community, communities against themselves and allowed fore..."
...,...,...,...,...,...
148945,https://www.aljazeera.com/news/2021/4/8/moons-ruling-party-dealt-crushing-poll-defeats-in-south-korea,2021-04-08T03:46:20,South Korea: Moon’s party dealt crushing defeats in mayoral polls,Conservative opposition won landslide victories for 